### DESEQ2 analysis of AD/PD dataset

In [1]:
rm(list=ls())
#load necessary libraries 
library(ggplot2)
library(DESeq2)
library("BiocParallel")
parallelFlag=TRUE
register(MulticoreParam(50))
library("IHW")
library("pheatmap")
library(sva)
library(limma)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,
    unsplit, which, which.max, which

## Load data and design

In [2]:
#load ATAC-seq raw read counts
data=read.table('../adpd.atac.idr.counts.txt.gz',header=TRUE,sep='\t')
#concatenate chrom/start/end columns values to server as rownames for the dataframe of the form chrom_start_end 
rownames(data)=paste(data$chrom,data$start,data$end,sep="_")
data$chrom=NULL
data$start=NULL
data$end=NULL

data=data[rowSums(data)>0,]


In [3]:
#load the metadata
batches=read.table("../batches.filtered.csv",header=TRUE,sep='\t')

## Perform analysis on TypeMod & RegionMod

In [4]:
Grouping <- factor(paste0(batches$Cohort,".",batches$RegionMod, ".", batches$TypeMod))
batches$Grouping=Grouping

In [5]:
#SVA can't handle NA values, so we have no choice but to interpolate to the mode for missing entries in PMI & ApoE 
batches$ApoE[is.na(batches$ApoE)]='3_3'
batches$PMI[is.na(batches$PMI)]=mean(na.omit(batches$PMI))

## Create the DESeq2 Object

In [6]:
#Create DESeq object
dds <- DESeqDataSetFromMatrix(countData = data,
                              colData = batches,
                              design = ~Grouping+Gender+expired_age+PMI+ApoE)#+TissueCenter +Batch
#TissueCenter and Batch are confounded


  the design formula contains a numeric variable with integer values,
  specifying a model with increasing fold change for higher values.
  did you mean for this to be a factor? if so, first convert
  this variable to a factor using the factor() function


## Differential Accessibility Operation

In [7]:
#Run the differential analysis
dds <- DESeq(dds,parallel = TRUE)

estimating size factors
estimating dispersions
gene-wise dispersion estimates: 50 workers
mean-dispersion relationship
final dispersion estimates, fitting model and testing: 50 workers


## Standard BH Correction (no optimal thresholding) 

In [8]:
res=results(dds)
summary(res)

res=results(dds,independentFiltering=FALSE)
summary(res)

res=results(dds,filterFun = ihw)
summary(res)


out of 385725 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 1216, 0.32%
LFC < 0 (down)     : 2535, 0.66%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results


out of 385725 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 1216, 0.32%
LFC < 0 (down)     : 2535, 0.66%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results


out of 385725 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 1062, 0.28%
LFC < 0 (down)     : 9967, 2.6%
outliers [1]       : 0, 0%
[1] see 'cooksCutoff' argument of ?results
see metadata(res)$ihwResult on hypothesis weighting



In [9]:
resultsNames(dds)

[1] "Intercept"                            
 [2] "Grouping_AD.CAUD.CTRL_vs_AD.CAUD.ADAD"
 [3] "Grouping_AD.CAUD.LOAD_vs_AD.CAUD.ADAD"
 [4] "Grouping_AD.HIPP.ADAD_vs_AD.CAUD.ADAD"
 [5] "Grouping_AD.HIPP.CTRL_vs_AD.CAUD.ADAD"
 [6] "Grouping_AD.HIPP.LOAD_vs_AD.CAUD.ADAD"
 [7] "Grouping_AD.PARL.ADAD_vs_AD.CAUD.ADAD"
 [8] "Grouping_AD.PARL.CTRL_vs_AD.CAUD.ADAD"
 [9] "Grouping_AD.PARL.LOAD_vs_AD.CAUD.ADAD"
[10] "Grouping_AD.SMTG.ADAD_vs_AD.CAUD.ADAD"
[11] "Grouping_AD.SMTG.CTRL_vs_AD.CAUD.ADAD"
[12] "Grouping_AD.SMTG.LOAD_vs_AD.CAUD.ADAD"
[13] "Grouping_PD.CAUD.ADPD_vs_AD.CAUD.ADAD"
[14] "Grouping_PD.CAUD.CTRL_vs_AD.CAUD.ADAD"
[15] "Grouping_PD.CAUD.LOPD_vs_AD.CAUD.ADAD"
[16] "Grouping_PD.HIPP.ADPD_vs_AD.CAUD.ADAD"
[17] "Grouping_PD.HIPP.CTRL_vs_AD.CAUD.ADAD"
[18] "Grouping_PD.HIPP.LOPD_vs_AD.CAUD.ADAD"
[19] "Grouping_PD.MDFG.ADPD_vs_AD.CAUD.ADAD"
[20] "Grouping_PD.MDFG.CTRL_vs_AD.CAUD.ADAD"
[21] "Grouping_PD.MDFG.LOPD_vs_AD.CAUD.ADAD"
[22] "Grouping_PD.PTMN.ADPD_vs_AD.CAUD.ADAD"
[23] "Grouping_PD.PTMN.CTRL_vs_AD.CAUD.ADAD"
[24] "Grouping_PD.PTMN.LOPD_vs_AD.CAUD.ADAD"
[25] "Grouping_PD.SMTG.ADPD_vs_AD.CAUD.ADAD"
[26] "Grouping_PD.SMTG.CTRL_vs_AD.CAUD.ADAD"
[27] "Grouping_PD.SMTG.LOPD_vs_AD.CAUD.ADAD"
[28] "Grouping_PD.SUNI.ADPD_vs_AD.CAUD.ADAD"
[29] "Grouping_PD.SUNI.CTRL_vs_AD.CAUD.ADAD"
[30] "Grouping_PD.SUNI.LOPD_vs_AD.CAUD.ADAD"
[31] "Gender_Male_vs_Female"                
[32] "expired_age"                          
[33] "PMI"                                  
[34] "ApoE_2_3_vs_2_2"                      
[35] "ApoE_3_3_vs_2_2"                      
[36] "ApoE_3_4_vs_2_2"                      
[37] "ApoE_4_4_vs_2_2"

In [10]:
group1=c('AD.CAUD.ADAD',
        'AD.CAUD.ADAD',
        'AD.HIPP.ADAD',
        'AD.HIPP.ADAD',
        'AD.PARL.ADAD',
        'AD.PARL.ADAD',
        'AD.SMTG.ADAD',
        'AD.SMTG.ADAD')
group2=c('AD.CAUD.LOAD',
        'AD.CAUD.CTRL',
        'AD.HIPP.LOAD',
        'AD.HIPP.CTRL',
        'AD.PARL.LOAD',
        'AD.PARL.CTRL',
        'AD.SMTG.LOAD',
        'AD.SMTG.CTRL')


In [11]:
comparisons=c("ad_caud_adpd_vs_load",
    "ad_caud_adad_vs_ctrl",
    "ad_hipp_adad_vs_load",
    "ad_hipp_adad_vs_ctrl",
    "ad_parl_adad_vs_load",
    "ad_parl_adad_vs_ctrl",
    "ad_smtg_adad_vs_load",
    "ad_smtg_adad_vs_ctrl")


In [12]:
pval_thresh=0.05
lfc_thresh=1

In [13]:
##get the results for the various contrasts 
numcomparisons=length(comparisons)
for(i in seq(1,numcomparisons))
{
 res=results(dds, contrast=c("Grouping", group1[i],group2[i]),parallel=TRUE)
 res$logPadj=-1*log10(res$padj)
 res=as.data.frame(res)
 res=na.omit(res)
 res$sig=res$padj<=pval_thresh & abs(res$log2FoldChange)>lfc_thresh
    
 #extract the differential peaks 
 sigsubset=res[res$sig==TRUE,]
 sig=nrow(sigsubset)
 up=sum(sigsubset$log2FoldChange>0)
 down=sum(sigsubset$log2FoldChange<0)
 curtitle=paste(comparisons[i],'\n','sig:',sig,'\n','up:',up,'\n','down:',down,'\n')   
 print(curtitle)
    
 #generate a volcano plot 
 png(paste("volcano_diff",comparisons[i],".png",sep=""))
 print(ggplot(data=res,
               aes(y=res$logPadj,x=res$log2FoldChange,color=res$sig))+
               geom_point(alpha=0.1)+
               xlab("log2(FC)")+
               ylab("-log10(pval)")+
               theme_bw()+
               scale_color_manual(values=c("#000000","#FF0000"))+
               ggtitle(curtitle))
  dev.off() 
  #write differential peaks to a TSV file 
  write.table(sigsubset,file=paste("diff_",comparisons[i],".tsv",sep=""),
              quote=FALSE,sep='\t',row.names = TRUE,col.names = TRUE)
}

[1] "ad_caud_adpd_vs_load \n sig: 127588 \n up: 42134 \n down: 85454 \n"
[1] "ad_caud_adad_vs_ctrl \n sig: 132600 \n up: 43771 \n down: 88829 \n"
[1] "ad_hipp_adad_vs_load \n sig: 13580 \n up: 3548 \n down: 10032 \n"
[1] "ad_hipp_adad_vs_ctrl \n sig: 35325 \n up: 9072 \n down: 26253 \n"
[1] "ad_parl_adad_vs_load \n sig: 11329 \n up: 9676 \n down: 1653 \n"
[1] "ad_parl_adad_vs_ctrl \n sig: 28357 \n up: 19799 \n down: 8558 \n"
[1] "ad_smtg_adad_vs_load \n sig: 9368 \n up: 4183 \n down: 5185 \n"
[1] "ad_smtg_adad_vs_ctrl \n sig: 20260 \n up: 12384 \n down: 7876 \n"


## Repeat analysis with Type and Region 

In [14]:
Grouping <- factor(paste0(batches$Cohort,".",batches$Region, ".", batches$Type))
batches$Grouping=Grouping

In [15]:
#Create DESeq object
dds2 <- DESeqDataSetFromMatrix(countData = data,
                              colData = batches,
                              design = ~Grouping+Gender+expired_age+PMI+ApoE)#TissueCenter +Batch

  the design formula contains a numeric variable with integer values,
  specifying a model with increasing fold change for higher values.
  did you mean for this to be a factor? if so, first convert
  this variable to a factor using the factor() function


In [ ]:
#Run the differential analysis
dds2 <- DESeq(dds2,parallel = TRUE)

estimating size factors
estimating dispersions
gene-wise dispersion estimates: 50 workers


In [ ]:
comparisons=c("ad_caud_adpd_vs_load",
    "ad_caud_adad_vs_ctrl",
    "ad_caud_adad_vs_ctrh",
    "ad_hipp_adad_vs_load",
    "ad_hipp_adad_vs_ctrl",
    "ad_hipp_adad_vs_ctrh",
    "ad_parl_adad_vs_load",
    "ad_parl_adad_vs_ctrl",
    "ad_parl_adad_vs_ctrh",
    "ad_smtg_adad_vs_load",
    "ad_smtg_adad_vs_ctrl",
    "ad_smtg_adad_vs_ctrh")


In [ ]:
group1=c('AD.CAUD.ADAD',
        'AD.CAUD.ADAD',
        'AD.CAUD.ADAD',
        'AD.HIPP.ADAD',
        'AD.HIPP.ADAD',
        'AD.HIPP.ADAD',
        'AD.PARL.ADAD',
        'AD.PARL.ADAD',
        'AD.PARL.ADAD',
        'AD.SMTG.ADAD',
        'AD.SMTG.ADAD',
        'AD.SMTG.ADAD')
group2=c('AD.CAUD.LOAD',
        'AD.CAUD.CTRL',
        'AD.CAUD.CTRH',
        'AD.HIPP.LOAD',
        'AD.HIPP.CTRL',
        'AD.HIPP.CTRH',
        'AD.PARL.LOAD',
        'AD.PARL.CTRL',
        'AD.PARL.CTRH',
        'AD.SMTG.LOAD',
        'AD.SMTG.CTRL',
        'AD.SMTG.CTRH')


In [ ]:
##get the results for the various contrasts 
numcomparisons=length(comparisons)
for(i in seq(1,numcomparisons))
{
 res=results(dds2, contrast=c("Grouping", group1[i],group2[i]),parallel=TRUE)
 res$logPadj=-1*log10(res$padj)
 res=as.data.frame(res)
 res=na.omit(res)
 res$sig=res$padj<=pval_thresh & abs(res$log2FoldChange)>lfc_thresh
    
 #extract the differential peaks 
 sigsubset=res[res$sig==TRUE,]
 sig=nrow(sigsubset)
 up=sum(sigsubset$log2FoldChange>0)
 down=sum(sigsubset$log2FoldChange<0)
 curtitle=paste(comparisons[i],'\n','sig:',sig,'\n','up:',up,'\n','down:',down,'\n')   
 print(curtitle)
    
 #generate a volcano plot 
 png(paste("expanded_volcano_diff",comparisons[i],".png",sep=""))
 print(ggplot(data=res,
               aes(y=res$logPadj,x=res$log2FoldChange,color=res$sig))+
               geom_point(alpha=0.1)+
               xlab("log2(FC)")+
               ylab("-log10(pval)")+
               theme_bw()+
               scale_color_manual(values=c("#000000","#FF0000"))+
               ggtitle(curtitle))
  dev.off() 
  #write differential peaks to a TSV file 
  write.table(sigsubset,file=paste("expanded_diff_",comparisons[i],".tsv",sep=""),
              quote=FALSE,sep='\t',row.names = TRUE,col.names = TRUE)
}

In [22]:
#store dds object so it can be loaded readily in the future 
save(dds,dds2, file = "AD.DESEQ2.model.noSVA.RData")